In [1]:
import pandas as pd
import numpy as np

In [2]:
from bs4 import BeautifulSoup as soup
from urllib.request import urlopen as uReq
import ssl
# This restores the same behavior as before.
context = ssl._create_unverified_context()
my_url="https://www.ebay.com/b/Apple/bn_21819543?_dmd=2&mag=1&rt=nc"
uClient = uReq(my_url, context=context)
page_html = uClient.read()
uClient.close()

In [3]:
page_soup = soup(page_html, "html.parser")
page_soup

<!DOCTYPE html>
<html class="brwweb" lang="en"><head><!--brwweb#s0-1-0-2--><link href="//ir.ebaystatic.com" rel="dns-prefetch"/><link href="//secureir.ebaystatic.com" rel="dns-prefetch"/><link href="//i.ebayimg.com" rel="dns-prefetch"/><link href="//rover.ebay.com" rel="dns-prefetch"/><script>$ssgST=new Date().getTime();</script><!--brwweb/--><meta charset="utf-8"/><meta content="ie=edge" http-equiv="x-ua-compatible"/><meta content="width=device-width, initial-scale=1, maximum-scale=1" name="viewport"/><!-- SEO METADATA START --><meta content="ebay-objects:ecommerce" property="og:type"><link href="https://ngebay.com/b/Apple/bn_21819543_pgn=1" rel="canonical"><meta content="Shop eBay for great deals on Apple. You'll find new or used products in Apple on eBay. Free shipping on selected items." name="description"><meta content="https://ngebay.com/b/Apple/bn_21819543?_pgn=1" property="og:url"><link href="https://ngebay.com/b/Apple/bn_21819543?_pgn=null" rel="next"><meta content="Shop eBay 

In [4]:
containers = page_soup.findAll("section", { "class": "brw-river bwrvr"})
print(len(containers))

1


In [5]:
print(soup.prettify(containers[0]))

<section class="brw-river bwrvr" data-viewport='{"trackableId":"01JPEVPW6F2HDXWJGZCC1EZ6AX"}'>
 <!--F#1-->
 <ul class="brwrvr__item-results brwrvr__item-results--gallery">
  <li class="brwrvr__item-card brwrvr__item-card--gallery">
   <div class="brwrvr__item-card__body">
    <div class="brwrvr__item-card__wrapper">
     <div class="brwrvr__item-card__image-wrapper brwrvr__item-card__image-wrapper--carousel">
      <div aria-label="New iPad Pencil for Apple iPad Air/iPad Mini/iPad Pro | Wireless Charging | PS2 - Image 1 of 4" aria-roledescription="Carousel" class="carousel brwrvr__item-card__carousel" role="group">
       <div class="carousel__container" id="s0-1-0-15-4-49[3]-18[6]-0-1-2-1[0]-6-2-container">
        <button aria-disabled="true" aria-label="Go to previous image" class="carousel__control carousel__control--prev" type="button">
         <svg aria-hidden="true" class="icon icon--16" focusable="false">
          <use href="#icon-chevron-left-16">
          </use>
         <

In [6]:
containers[0].findAll("img", {"class": "brwrvr__item-card__image"})[0]["src"]

'https://ir.ebaystatic.com/cr/v/c1/s_1x2.gif'

In [7]:
containers[0].findAll("h3", {"class": "textual-display bsig__title__text"})[1].text.strip()

'Apple iPhone 11 64GB Black Unlocked Excellent Condition'

In [8]:
containers[0].findAll("span", {"class": "textual-display bsig__price bsig__price--displayprice"})[0].text.strip()

'$24.99'

In [9]:
containers[0].findAll("span", {"class": "textual-display bsig__generic bsig__logisticsCost"})[0].text.strip()

'Free shipping'

In [10]:
containers[0].findAll("span", {"class": "textual-display bsig__product-review__count"})[0].text.strip()

'(24)'

In [11]:
containers[0].findAll("span", {"class": "textual-display bsig__generic bsig__deliveryOptions"})[0].text.strip()

'Est. delivery Wed, Mar 19'

In [12]:
containers[0].findAll("span", {"class": "textual-display bsig__generic bsig__deliveryOptions bold"})[0].text.strip()

'Wed, Mar 19'

In [30]:
# Now that we are able to extract details for one product, let's now loop for other products
import numpy as np
products_image = []
products_name = []
pricing = []
# products_rating = []
delivery_method = []
delivery_review = []

for product in containers[0:]:
    # Let's scrape the product image
    image = product.findAll("img", {"class": "brwrvr__item-card__image"})
    if len(image) == 0:
        products_image.append(np.nan)
    else:
        products_image.append(image[0]['src'])
    
    # Let's scrape the product name
    name = product.findAll("h3", {"class": "textual-display bsig__title__text"})
    if len(name) == 0:
        products_name.append(np.nan)
    else:
        products_name.append(name[0].text.strip())

    # # scrape rating
    # rating= product.findAll("span", {"class": "textual-display bsig__product-review__count"})
    # if len(rating) == 0:
    #     products_rating.append(np.nan)
    # else:
    #     product_rating.append(rating[0].text.strip())

    # Let's crape the delivery method
    transport = product.findAll("span", {"class": "textual-display bsig__generic bsig__logisticsCost"})
    if len(transport) == 0:
        delivery_method.append(np.nan)
    else:
        delivery_method.append(transport[0].text.strip())

    # Now let's have a look at its price:
    price = product.findAll("span", {"class": "textual-display bsig__price bsig__price--displayprice"})
    if len(price) == 0:
        pricing.append(np.nan)
    else:
        pricing.append(price[0].text.strip())  
    
    # Now let's have a look at its delivery review:
    review = product.findAll("span", {"class": "textual-display bsig__generic bsig__deliveryOptions"})
    if len(review) == 0:
        delivery_review.append(np.nan)
    else:
        delivery_review.append(review[0].text.strip())
       
print(len(products_image))
print(len(products_name))
print(len(pricing))
# print(len(products_rating))
print(len(delivery_method))
print(len(delivery_review))

1
1
1
1
1


In [33]:
df = pd.DataFrame({"Product_Name":products_name, "Pricing":pricing, "Delivery_method":delivery_method,
                   "Delivery_review":delivery_review, "image":products_image})
df.to_csv("apple_products.csv")
tv_data = pd.read_csv("apple_products.csv")
tv_data.head()

,Unnamed: 0,Product_Name,Pricing,Delivery_method,Delivery_review,image
0,0,New iPad Pencil for Apple iPad Air/iPad Mini/i...,$24.99,Free shipping,"Est. delivery Wed, Mar 19",https://ir.ebaystatic.com/cr/v/c1/s_1x2.gif
1,1,Apple iPhone 13 Pro 128GB Unlocked - Excellent,$389.99,Free shipping,"Est. delivery Wed, Mar 19",https://ir.ebaystatic.com/cr/v/c1/s_1x2.gif
2,2,Apple iPhone 14 128GB Unlocked - Excellent,$364.95,Free shipping,"Est. delivery Wed, Mar 19",https://ir.ebaystatic.com/cr/v/c1/s_1x2.gif
3,3,2024 Apple MacBook Air 15-inch M3 Chip 16GB RA...,$949.00,Free shipping,"Est. delivery Wed, Mar 19",https://ir.ebaystatic.com/cr/v/c1/s_1x2.gif
4,4,Apple iPad Air 2 9.7in Tablet Choose Color 16G...,$79.99 to $89.99,Free shipping,"Est. delivery Wed, Mar 19",https://ir.ebaystatic.com/cr/v/c1/s_1x2.gif


In [31]:
import numpy as np
products_image = []
products_name = []
pricing = []
# products_rating = []
delivery_method = []
delivery_review = []

my_url=["https://www.ebay.com/b/Apple/bn_21819543",
        "https://www.ebay.com/b/Apple/bn_21819543?_pgn=2&rt=nc",
        "https://www.ebay.com/b/Apple/bn_21819543?_pgn=3&rt=nc",
        "https://www.ebay.com/b/Apple/bn_21819543?_pgn=4&rt=nc",
        "https://www.ebay.com/b/Apple/bn_21819543?_pgn=5&rt=nc",
        "https://www.ebay.com/b/Apple/bn_21819543?_pgn=5&rt=nc",
        "https://www.ebay.com/b/Apple/bn_21819543?_pgn=6&rt=nc",
        "https://www.ebay.com/b/Apple/bn_21819543?_pgn=7&rt=nc",
        "https://www.ebay.com/b/Apple/bn_21819543?_pgn=8&rt=nc",
        "https://www.ebay.com/b/Apple/bn_21819543?_pgn=9&rt=nc",
        "https://www.ebay.com/b/Apple/bn_21819543?_pgn=10&rt=nc",
       ]

for link in my_url:

    # This restores the same behavior as before.
    context = ssl._create_unverified_context()
    uClient = uReq(link, context=context)
    page_html = uClient.read()
    uClient.close()

    page_soup = soup(page_html, "html.parser")
    
    containers = page_soup.findAll("section", { "class": "brw-river bwrvr"})
    
    for product in containers[0:]:
    # Let's scrape the product image
        image = product.findAll("img", {"class": "brwrvr__item-card__image"})
        if len(image) == 0:
            products_image.append(np.nan)
        else:
            products_image.append(image[0]['src'])
    
    # Let's scrape the product name
        name = product.findAll("h3", {"class": "textual-display bsig__title__text"})
        if len(name) == 0:
            products_name.append(np.nan)
        else:
            products_name.append(name[0].text.strip())

    # # scrape rating
    # rating= product.findAll("span", {"class": "textual-display bsig__product-review__count"})
    # if len(rating) == 0:
    #     products_rating.append(np.nan)
    # else:
    #     product_rating.append(rating[0].text.strip())

    # Let's crape the delivery method
        transport = product.findAll("span", {"class": "textual-display bsig__generic bsig__logisticsCost"})
        if len(transport) == 0:
            delivery_method.append(np.nan)
        else:
            delivery_method.append(transport[0].text.strip())

    # Now let's have a look at its price:
        price = product.findAll("span", {"class": "textual-display bsig__price bsig__price--displayprice"})
        if len(price) == 0:
            pricing.append(np.nan)
        else:
            pricing.append(price[0].text.strip())  
    
    # Now let's have a look at its delivery review:
        review = product.findAll("span", {"class": "textual-display bsig__generic bsig__deliveryOptions"})
        if len(review) == 0:
            delivery_review.append(np.nan)
        else:
            delivery_review.append(review[0].text.strip())
       
print(len(products_image))
print(len(products_name))
print(len(pricing))
# print(len(products_rating))
print(len(delivery_method))
print(len(delivery_review))
    

11
11
11
11
11


In [32]:
import pandas as pd
df = pd.DataFrame({"Product_Name":products_name, "Pricing":pricing, "Delivery_method":delivery_method,
                   "Delivery_review":delivery_review, "image":products_image})
df.to_csv("apple_products.csv")
tv_data = pd.read_csv("apple_products.csv")
tv_data.head(11)

,Unnamed: 0,Product_Name,Pricing,Delivery_method,Delivery_review,image
0,0,New iPad Pencil for Apple iPad Air/iPad Mini/i...,$24.99,Free shipping,"Est. delivery Wed, Mar 19",https://ir.ebaystatic.com/cr/v/c1/s_1x2.gif
1,1,Apple iPhone 13 Pro 128GB Unlocked - Excellent,$389.99,Free shipping,"Est. delivery Wed, Mar 19",https://ir.ebaystatic.com/cr/v/c1/s_1x2.gif
2,2,Apple iPhone 14 128GB Unlocked - Excellent,$364.95,Free shipping,"Est. delivery Wed, Mar 19",https://ir.ebaystatic.com/cr/v/c1/s_1x2.gif
3,3,2024 Apple MacBook Air 15-inch M3 Chip 16GB RA...,$949.00,Free shipping,"Est. delivery Wed, Mar 19",https://ir.ebaystatic.com/cr/v/c1/s_1x2.gif
4,4,Apple iPad Air 2 9.7in Tablet Choose Color 16G...,$79.99 to $89.99,Free shipping,"Est. delivery Wed, Mar 19",https://ir.ebaystatic.com/cr/v/c1/s_1x2.gif
5,5,Apple iPad Air 2 9.7in Tablet Choose Color 16G...,$79.99 to $89.99,Free shipping,"Est. delivery Wed, Mar 19",https://ir.ebaystatic.com/cr/v/c1/s_1x2.gif
6,6,Apple iPhone 13 128GB Unlocked - Good,$284.99,Free shipping,"Est. delivery Wed, Mar 19",https://ir.ebaystatic.com/cr/v/c1/s_1x2.gif
7,7,Apple iPhone 12 Pro Max A2342 128GB Unlocked G...,$392.71,Free shipping,"Est. delivery Wed, Mar 19",https://ir.ebaystatic.com/cr/v/c1/s_1x2.gif
8,8,Original 61W 20.3V 3A USB-C Charger For Apple ...,$19.99,Free shipping,"Est. delivery Wed, Mar 19",https://ir.ebaystatic.com/cr/v/c1/s_1x2.gif
9,9,Apple iPhone 13 Pro Max 512GB Unlocked AT&T T-...,$479.95,Free shipping,"Est. delivery Wed, Mar 19",https://ir.ebaystatic.com/cr/v/c1/s_1x2.gif
